In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

import numpy as np
import pandas as pd
import xgboost as xgb
# remove warnings
import warnings

warnings.filterwarnings('ignore')
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error

# my own module
from features import data_utils
from readData import readData, data_preprocess

In [2]:
import numpy as np
import pandas as pd
import datetime
from sklearn.cross_validation import KFold
from sklearn.cross_validation import train_test_split
import time
#%matplotlib inline
from sklearn import preprocessing
from sklearn.preprocessing import Imputer
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import ShuffleSplit
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.linear_model import Ridge, LassoCV,LassoLarsCV, ElasticNet
from sklearn.kernel_ridge import KernelRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from scipy.stats import skew

def RMSLE_(y_val, y_val_pred):
    return np.sqrt(np.mean((np.log(y_val+1)-np.log(y_val_pred+1))**2))
RMSLE = make_scorer(RMSLE_, greater_is_better=False) 

def RMSE_(y_val, y_val_pred):
    return np.sqrt(np.mean((y_val-y_val_pred)**2))
RMSE = make_scorer(RMSE_, greater_is_better=False)

from time import time

In [3]:
X_train, X_test, y_train = readData(isLog1p=True)
#X_train, X_test = data_preprocess(X_train, X_test)

conbined_data: (36650, 702)


In [4]:
xgb_params = {
        'eta': 0.05,
        'max_depth': 5,
        'subsample': 0.7,
        'colsample_bytree': 0.7,
        'objective': 'reg:linear',
        'eval_metric': 'rmse',
        'silent': 1,
        'seed':5
    }

dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test)

cv_output = xgb.cv(xgb_params, dtrain, num_boost_round=2000, early_stopping_rounds=50,\
    verbose_eval=50, show_stdv=False)
cv_output[['train-rmse-mean', 'test-rmse-mean']].plot()

dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test)

num_boost_rounds = len(cv_output)
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=num_boost_rounds)

#fig, ax = plt.subplots(1, 1, figsize=(8, 13))
#xgb.plot_importance(model, max_num_features=50, height=0.5, ax=ax)

y_predict = model.predict(dtest)
#y_predict = np.exp(y_predict)-1
#y_predict = np.round(y_predict)#np.round(y_predict * 0.99)

[0]	train-rmse:14.4028	test-rmse:14.4029
[50]	train-rmse:1.15811	test-rmse:1.16151
[100]	train-rmse:0.310493	test-rmse:0.335799
[150]	train-rmse:0.283384	test-rmse:0.320504
[200]	train-rmse:0.272601	test-rmse:0.319036
[250]	train-rmse:0.263476	test-rmse:0.318562
[300]	train-rmse:0.254741	test-rmse:0.318513
[350]	train-rmse:0.247279	test-rmse:0.318398
[400]	train-rmse:0.240182	test-rmse:0.318391
[450]	train-rmse:0.23361	test-rmse:0.318726


In [4]:
train, test = data_preprocess(X_train, X_test)
xgb_params = {'learning_rate': 0.05,
              'max_depth': 4,
              'subsample': 0.95,
              'reg_alpha': 0,
              'min_child_weight':4,
              'colsample_bytree': 0.95,
              'gamma':.4,
              'objective': 'reg:linear',
              'silent': 1,
              #'booster' :'gbtree',
              #'tuneLength': 3,
              'seed': 5 }
#print(len(cv_output)) #309

xgb_params['n_estimators']=620 #len(cv_output)

model = XGBRegressor(**xgb_params)
model.fit(train, y_train)
y_predict = model.predict(test)

NameError: name 'dtest' is not defined

In [7]:
model = GradientBoostingRegressor(learning_rate=0.02, n_estimators=500,min_samples_leaf=70, 
                                 min_samples_split=200, max_features='sqrt',max_depth=6,subsample=0.85,
                                 random_state=10)
model.fit(train, y_train)
y_predict = model.predict(test)

In [11]:
et_params = {
    'n_jobs': 16,
    'n_estimators': 100,
    'max_features': 0.5,
    'max_depth': 12,
    'min_samples_leaf': 2,
    'random_state': 10
}
model = ExtraTreesRegressor(**et_params)
#train, test = data_preprocess(X_train, X_test)
model.fit(train, y_train)
y_predict = model.predict(test)

In [4]:
rf_params = {
    'n_jobs': 16,
    'n_estimators': 100,
    'max_features': 0.2,
    'max_depth': 12,
    'min_samples_leaf': 2,
}
model = RandomForestRegressor(**rf_params)
train, test = data_preprocess(X_train, X_test)
model.fit(train, y_train)
y_predict = model.predict(test)

In [4]:
rd_params = {'alpha': 1}
model = Ridge(**rd_params)
train, test = data_preprocess(X_train, X_test)
model.fit(train, y_train)
y_predict = model.predict(test)

In [5]:
y_predict = np.exp(y_predict)-1
#y_predict = np.round(y_predict)#np.round(y_predict * 0.99)
output = pd.DataFrame({'id': pd.read_csv('../../input/test.csv')['id'], 'price_doc': y_predict})
output.to_csv('../result/jun26_5_rd.csv', index=False)